<a href="https://colab.research.google.com/github/wairiukoirwine/E-citizen-ai/blob/main/ecitizen_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!pip install sentence-transformers faiss-cpu PyPDF2 gradio

import gradio as gr
import PyPDF2
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Global variables
pdf_paragraphs = []
pdf_embeddings = None
pdf_index = None

# Function to load PDF and process paragraphs
def upload_pdf(file):
    global pdf_paragraphs, pdf_embeddings, pdf_index
    reader = PyPDF2.PdfReader(file.name)
    pdf_paragraphs = []

    for page in reader.pages:
        text = page.extract_text()
        if text:
            # Split text into paragraphs by double newline
            pdf_paragraphs.extend([p.strip() for p in text.split("\n\n") if p.strip()])

    # Compute embeddings
    pdf_embeddings = model.encode(pdf_paragraphs, convert_to_tensor=False)
    pdf_embeddings_np = np.array(pdf_embeddings, dtype='float32')

    # Build FAISS index
    pdf_index = faiss.IndexFlatL2(pdf_embeddings_np.shape[1])
    pdf_index.add(pdf_embeddings_np)

    return "PDF uploaded and processed. Bot is ready to answer questions accurately."

# Function to answer user query
def answer_query(user_query, chat_history):
    global pdf_paragraphs, pdf_index
    if not pdf_paragraphs:
        chat_history.append((f"User: {user_query}", "Bot: Please upload the PDF first."))
        return "", chat_history

    # Encode query and search
    query_emb = model.encode([user_query], convert_to_tensor=False)
    D, I = pdf_index.search(np.array(query_emb, dtype='float32'), k=1)
    top_para = pdf_paragraphs[I[0][0]]

    chat_history.append((f"User: {user_query}", f"Bot: {top_para}"))
    return "", chat_history

# Gradio interface
with gr.Blocks() as demo:
    chat_history = gr.State([])

    gr.Markdown("## eCitizen Smart Q&A Bot (PDF-based)")

    with gr.Row():
        txt_input = gr.Textbox(label="Your Question", placeholder="Ask about eCitizen services...")
        send_btn = gr.Button("Send")

    upload_pdf_file = gr.File(label="Upload eCitizen PDF", file_types=[".pdf"])
    upload_status = gr.Textbox(label="Upload Status")

    chat_display = gr.Chatbot(label="Chat History")

    send_btn.click(answer_query, [txt_input, chat_history], [txt_input, chat_display])
    upload_pdf_file.upload(upload_pdf, upload_pdf_file, upload_status)

demo.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-3110338792.py:67: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_display = gr.Chatbot(label="Chat History")
/tmp/ipython-input-3110338792.py:67: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chat_display = gr.Chatbot(label="Chat History")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3c216cf82186996b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
